In [1]:
from discord.ext import commands
import discord
import pickle
import csv
import os
import numpy as np
import scipy.stats as sps


In [165]:
def elo_func(kost, kills, deaths, win):
    kost_mu = 0.55
    kost_sigma = 0.2
    dist = sps.norm(loc=kost_mu, scale=kost_sigma)
    p = dist.cdf(kost + ((kost - 0.55) ** 2) / 7 + (kost - 0.55) / 9)

    if int(kost) == 1:
        kost_factor = 2
    else:    
        kost_factor = (np.sqrt(2) - 2 + 2 * (1 - np.sqrt(2)) * p) ** 2

    if win:
        return round(150 * kost_factor + kills * 10 - deaths * 10)
    else:
        return round(-100 * (2 + 0.343265 - kost_factor) + kills * 10 - deaths * 10)

# print('KOST  --  K MULTIPLIER  --  Win Elo')

# kost = 0.00
# for i in range(200):
#     kost_mu = 0.55
#     kost_sigma = 0.2
#     dist = sps.norm(loc=kost_mu, scale=kost_sigma)
#     p = dist.cdf(kost + ((kost - 0.55) ** 2) / 7 + (kost - 0.55) / 9)

#     if int(kost) == 1:
#         kost_factor = 2
#     else:    
#         kost_factor = (np.sqrt(2) - 2 + 2 * (1 - np.sqrt(2)) * p) ** 2
#     print('{:.2f}         x {:.5f}                  {}'.format(kost, kost_factor, elo_func(kost, 0, 0, True)))
#     kost += 0.05
#     if kost > 1.04:
#         break

KOST  --  K MULTIPLIER  --  Win Elo
0.00         x 0.34534                  52
0.05         x 0.34770                  52
0.10         x 0.35218                  53
0.15         x 0.36036                  54
0.20         x 0.37456                  56
0.25         x 0.39812                  60
0.30         x 0.43544                  65
0.35         x 0.49190                  74
0.40         x 0.57332                  86
0.45         x 0.68466                  103
0.50         x 0.82797                  124
0.55         x 1.00000                  150
0.60         x 1.19073                  179
0.65         x 1.38440                  208
0.70         x 1.56332                  234
0.75         x 1.71304                  257
0.80         x 1.82621                  274
0.85         x 1.90342                  286
0.90         x 1.95091                  293
0.95         x 1.97726                  297
1.00         x 2.00000                  300
